In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [3]:
data_file = 'repressor_timeseries.csv'
df_data = pd.read_csv(data_file, sep='\t')

print('Dataset:')
print(df_data.head())

num_experiments = 3

#Get a 1D numpy array of x-values (repressor concentrations)
x = np.ravel(np.array(list(df_data.repressor_conc)))[::-1]
#Get a 2D matrix of measured target y-values (fluorescence). Rows are indepdendent experiments, Columns fluorescence values.
targets = np.zeros((num_experiments, len(x)))
for i in range(1, 1 + num_experiments) :
    target = np.ravel(np.array(list(df_data['fluorescence_' + str(i)])))[::-1]
    targets[i-1, :] = target

Dataset:
   repressor_conc  fluorescence_1  fluorescence_2  fluorescence_3
0       41.152263            1683            1811            2139
1       13.717421            1818            1724            1789
2        4.572474            1863            1878            1909
3        1.524158            2301            2264            2096
4        0.508053            2329            2245            2610


In [6]:
print(targets)


# fig, ax = plt.subplots(1)
# ax.semilogx()

[[20962. 20877. 20624. 16697.  7718.  7059.  3686.  2936.  3330.  2329.
   2301.  1863.  1818.  1683.]
 [21047. 20292. 19405. 15775.  8933.  6300.  3539.  2796.  3133.  2245.
   2264.  1878.  1724.  1811.]
 [22735. 21919. 21047. 15395.  7321.  5952.  3343.  2865.  2774.  2610.
   2096.  1909.  1789.  2139.]]


In [ ]:
###
#
# Problem 2.a: Compute the mean and standard deviation of the measured values (mean fluorescence and deviation between the experiments,
# for each data point), and plot them. Where in the hill-curve, according to the data, do you see most variation?
#
###


In [ ]:
###
#
# Problem 2.b: Implement the function hill_curve,
# which computes the hill-curve values of the input repressor values x (numpy array).
#
###
def hill_curve(F_max, n, K, F_0, x) :
    """Hill Curve function

    Args:
        F_max (float): maximum expression level
        n (int): Hill coefficient
        K (float): dissociation constant
        F_0 (float): leaky expression
        x (float): repressor concentration

    Returns:
        float: current expression level
    """

    # return vector of expression levels for each repressor concentration
    F_x = np.zeros(len(x))

    return F_x

###
#
# Problem 2.b: Implement the function objective_function.
# It should return the Sum-of-squared Error between target (i.e. measured) and predicted fluorescence.
# Note that in the documentation for scipy.optimize.minimize, the objective function is called "fun",
# and takes (x, *args) as arguments. The x of fun(x,*args) is equivalent to "coeff_vec", and is a 1-D array 
# containing the parameters of the function to fit: F_max, n, K, F_0 for the hill_curve function.
# 
# The *args of fun(x,*args) is a tuple of arguments. In this case, the first element of args will be the 
# input repressor values ("x"), and the second element will be the target fluorescence values ("F_x").
# The predicted fluorescence values should be computed by calling the hill_curve function you defined above.
#
#
###
def objective_function(coeff_vec, args) :
    
    x, F_x = args
    
    F_max = coeff_vec[0]
    n = coeff_vec[1]
    K = coeff_vec[2]
    F_0 = coeff_vec[3]

    return 0

###
# This defines the constraints that must be satisfied by the numerical optimizer:
#
# F_max >= 0
# 0 <= n <= 5
# K >= 0
# F_0 >= 0
#
###
cons = (
    #F_max >= 0
    {'type': 'ineq',
    'fun' : lambda x: np.array([x[0]]),
    'jac' : lambda x: np.array([1, 0, 0, 0])},
    #n >= 0
    {'type': 'ineq',
    'fun' : lambda x: np.array([x[1]]),
    'jac' : lambda x: np.array([0, 1, 0, 0])},
    #n <= 5
    {'type': 'ineq',
    'fun' : lambda x: np.array([5 - x[1]]),
    'jac' : lambda x: np.array([0, -1, 0, 0])},
    #K >= 0
    {'type': 'ineq',
    'fun' : lambda x: np.array([x[2]]),
    'jac' : lambda x: np.array([0, 0, 1, 0])},
    #F_0 >= 0
    {'type': 'ineq',
    'fun' : lambda x: np.array([x[3]]),
    'jac' : lambda x: np.array([0, 0, 0, 1])}
)


###
#
# Problem 2.b: Write code using the scipy function scipy.optimize.minimize
# to fit a hill-curve model (consisting of the 4 parameters above) to the experimental data.
# The input values are the repressor concentration values "x".
# The target (measured) values are the mean fluorescence values "F_x".
# Finally plot the fitted hill-curve in the range (0, 100]. Overlay the experimental means.
#
# Tip: The hill-curve is a highly non-linear function of the free parameters we want to fit.
# In such cases, the optimization is not guaranteed to be convex and you could end up in a bad local minima.
# Therefore it's good practice to randomly initialize the parameters.
# For example, randomly initializing n in range [0, 5] and the other parameters in range [0, 10^5].
#
###





